# Calculate BIC curves

This notebook will create the file BICs2-30.obj, required to reproduce Figure 2 from *Heuristic Methods for Determining the Number of Classes in Unsupervised Classification of Climate Models*, E. Boland et al. 2022 (doi to follow). This requires cluster_utils.py and input datafiles via the googleapi CMIP6 store (see cluster_utils.py for more info)

Please attribute any plots or code from this notebook using the DOI from Zenodo: to come

Updated Nov 2022
E Atkinson & E Boland [emmomp@bas.ac.uk](email:emmomp@bas.ac.uk)

In [1]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:46506")
client

<Client: 'tcp://127.0.0.1:46506' processes=6 threads=6, memory=48.00 GiB>

In [2]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import os
import pickle
import cluster_utils as flt

### Load/create mask
Uncomment the first two lines to generate the file mask.npy if required

In [4]:
#data = flt.retrieve_profiles(timeRange = slice('1995-01', '1995-02'))
#np.save('data/mask', data['n'])
mask = np.load('data/mask.npy', allow_pickle=True)

In [1]:
ids = ['r1i1p1f2', 'r2i1p1f2', 'r3i1p1f2'] # Ensemble members to use
BICs = {}
model_folder = 'model'

### Fit 2-30 class models for each ensemble member
Saves each individual PCA model, GMM model and BIC curve to \[model_folder\]

Saves all BIC curves to \[model_folder\]/BICs2-30.obj

In [ ]:
for m_id in ids:
    path_id = '{}/{}'.format(model_folder, m_id)
    if not os.path.isdir(path_id):
        os.makedirs(path_id)
    print('Starting {}'.format(m_id))
    options = {'memberId' : m_id}
    
    # Load all 30 years of data
    data = flt.retrieve_profiles(timeRange = slice('1965-01', '1994-12'), mask=mask, options=options)
    
    # Create training dataset by taking random subsample of 7000 Southern Ocean profiles per month
    data_sampled = flt.random_sample(data, 7000).compute()
    data_sampled = flt.normalise_data(data_sampled, 'N') #Normalise
    
    pca = flt.train_pca(data_sampled, 3) #Fit PCA model    
    data_trans = flt.pca_transform(data_sampled, pca).compute() # Transform training set to PCA space
    
    bic = np.zeros(30)
    
    with open('{}/pca.obj'.format(path_id), 'wb') as file:
        pickle.dump(pca, file)
        file.close()
        
    print('Finished setup for {}'.format(m_id))
    
    for n_classes in range(2, 30):
        
        path_n = '{}/{}/{}'.format(model_folder, m_id, n_classes)
        if not os.path.isdir(path_n):
            os.makedirs(path_n)
        elif os.path.exists('{}/bic.obj'.format(path_n)):
            with open('{}/bic.obj'.format(path_n), 'rb') as file:
                bic[n_classes] = pickle.load(file)
                file.close()
            continue
        
        gmm = flt.train_gmm(data_trans, n_classes)
        bic[n_classes] = gmm.bic(data_trans)
        
        with open('{}/gmm.obj'.format(path_n), 'wb') as file:
            pickle.dump(gmm, file)
            file.close()
            
        with open('{}/bic.obj'.format(path_n), 'wb') as file:
            pickle.dump(bic[n_classes], file)
            file.close()
        
        print('Finished {} with {} classes'.format(m_id, n_classes))
        
    BICs[m_id] = bic
    
with open('{}/BICs2-30.obj'.format(model_folder), 'wb') as file:
    pickle.dump(BICs, file)
    file.close()

print('Done!')

Starting r1i1p1f2
Finished setup for r1i1p1f2
Finished r1i1p1f2 with 2 classes
Finished r1i1p1f2 with 3 classes
Finished r1i1p1f2 with 4 classes
Finished r1i1p1f2 with 5 classes
Finished r1i1p1f2 with 6 classes
Finished r1i1p1f2 with 7 classes
Finished r1i1p1f2 with 8 classes
